[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/intel-analytics/BigDL/blob/main/python/friesian/colab-notebook/examples/basic_ranking.ipynb) &nbsp;![](../../../image/GitHub-Mark-32px.png)[View source on GitHub](https://github.com/intel-analytics/BigDL/blob/main/python/friesian/colab-notebook/examples/basic_ranking.ipynb)

In [ ]:
#
# Copyright 2016 The BigDL Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
#
# Copyright 2020 The TensorFlow Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# This example is based on Tensorflow Recommenders example [basic ranking](https://www.tensorflow.org/recommenders/examples/basic_ranking).
# 

# Basic Ranking Example

In this tutorial, we're going to:

1. Use Friesian FeatureTable to get and preprocess the movielens data and split it into a training and test set.
2. Convert the preprocessed FeatureTable to an Orca TF Dataset and do some further data preprocessing.
3. Fit and evaluate the TFRS ranking model using Orca TF Estimator and Orca TF Dataset.

# Environment Preparation

### Install Java 8

Run the cell on the **Google Colab** to install jdk 1.8.

**Note**: if you run this notebook on your computer, root permission is required when running the cell to install Java 8. (You may ignore this cell if Java 8 has already been set up in your computer).

In [ ]:
# Install jdk8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
# Set environment variable JAVA_HOME.
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

### Install BigDL Friesian

You can install the latest pre-release version using pip install --pre --upgrade bigdl-friesian[train].

In [1]:
# Install latest pre-release version of BigDL Friesian 
# Installing BigDL Friesian from pip will automatically install pyspark and their dependencies.
!pip install --pre --upgrade bigdl-friesian[train]

     |████████████████████████████████| 183 kB 202 kB/s eta 0:00:01
     |████████████████████████████████| 23.9 MB 8.0 MB/s eta 0:00:01
  Using cached bigdl_tf-0.14.0.dev1-py3-none-manylinux2010_x86_64.whl (71.0 MB)
     |████████████████████████████████| 51.8 MB 7.4 MB/s eta 0:00:012     |██████████████████▌             | 30.0 MB 12.2 MB/s eta 0:00:02
  Using cached bigdl_math-0.14.0.dev1-py3-none-manylinux2010_x86_64.whl (35.4 MB)
  Using cached conda_pack-0.3.1-py2.py3-none-any.whl (27 kB)
     |████████████████████████████████| 51.0 MB 7.1 MB/s eta 0:00:012    |██████████████▏                 | 22.5 MB 1.7 MB/s eta 0:00:17
     |████████████████████████████████| 281 kB 201 kB/s eta 0:00:01
  Using cached aioredis-1.3.1-py3-none-any.whl (65 kB)
  Using cached ray-1.9.2-cp37-cp37m-manylinux2014_x86_64.whl (57.6 MB)
  Using cached hiredis-2.0.0-cp37-cp37m-manylinux2010_x86_64.whl (85 kB)
  Using cached aiohttp-3.8.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86

     |████████████████████████████████| 225 kB 8.8 MB/s eta 0:00:01
     |████████████████████████████████| 225 kB 9.4 MB/s eta 0:00:01
     |████████████████████████████████| 224 kB 10.9 MB/s eta 0:00:01
     |████████████████████████████████| 223 kB 10.4 MB/s eta 0:00:01
     |████████████████████████████████| 220 kB 6.9 MB/s eta 0:00:01
  Using cached redis-4.1.4-py3-none-any.whl (175 kB)
  Using cached Deprecated-1.2.13-py2.py3-none-any.whl (9.6 kB)
     |████████████████████████████████| 75 kB 459 kB/s eta 0:00:011
  Using cached idna-3.3-py3-none-any.whl (61 kB)
     |████████████████████████████████| 114 kB 269 kB/s eta 0:00:01
     |████████████████████████████████| 211 kB 595 kB/s eta 0:00:01
     |████████████████████████████████| 160 kB 9.0 MB/s eta 0:00:01
  Using cached rsa-4.8-py3-none-any.whl (39 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
     |████████████████████████████████| 138 kB 303

In [3]:
# Install required dependencies
!pip install tensorflow tensorflow-recommenders

     |████████████████████████████████| 511.7 MB 16 kB/s  eta 0:00:011    |█▏                              | 19.2 MB 10.7 MB/s eta 0:00:47     |████████▊                       | 138.7 MB 9.0 MB/s eta 0:00:42     |████████████████▎               | 259.5 MB 8.2 MB/s eta 0:00:31     |███████████████████▍            | 310.0 MB 9.3 MB/s eta 0:00:22     |█████████████████████▌          | 343.2 MB 8.7 MB/s eta 0:00:20     |██████████████████████▎         | 356.0 MB 10.7 MB/s eta 0:00:15     |████████████████████████████████| 511.1 MB 8.0 MB/s eta 0:00:01
  Using cached tensorflow_recommenders-0.6.0-py3-none-any.whl (85 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     |████████████████████████████████| 5.8 MB 10.2 MB/s eta 0:00:01
     |████████████████████████████████| 123 kB 9.0 MB/s eta 0:00:01
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached libclang-14.0.1-py2.py3-none-manylinux1_x86_64.whl (

In [4]:
import tensorflow as tf
print(tf.__version__)

2022-06-13 21:34:09.738528: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/d1/Dockers/grpcwnd/lib
2022-06-13 21:34:09.738555: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.9.1


## Distributed TFRS using Orca and Friesian APIs



In [5]:
import os
import math
import tensorflow as tf
import tensorflow_recommenders as tfrs

from bigdl.friesian.models import TFRSModel
from bigdl.orca import init_orca_context, stop_orca_context
from bigdl.orca import OrcaContext
from bigdl.friesian.feature import FeatureTable
from bigdl.orca.learn.tf2 import Estimator
from bigdl.orca.data.tf.data import Dataset

/home/yina/anaconda3/envs/py37/lib/python3.7/site-packages/bigdl/dllib/utils/engine.py:47: UserWarning: Find both SPARK_HOME and pyspark. You may need to check whether they match with each other. SPARK_HOME environment variable is set to: /opt/spark-2.4.3-bin-hadoop2.7, and pyspark is found in: /home/yina/anaconda3/envs/py37/lib/python3.7/site-packages/pyspark/__init__.py. If they are unmatched, please use one source only to avoid conflict. For example, you can unset SPARK_HOME and use pyspark only.
  warnings.warn(warning_msg)
/home/yina/anaconda3/envs/py37/lib/python3.7/site-packages/bigdl/dllib/utils/zoo_engine.py:47: UserWarning: Find both SPARK_HOME and pyspark. You may need to check whether they match with each other. SPARK_HOME environment variable is set to: /opt/spark-2.4.3-bin-hadoop2.7, and pyspark is found in: /home/yina/anaconda3/envs/py37/lib/python3.7/site-packages/pyspark/__init__.py. If they are unmatched, you are recommended to use one source only to avoid conflict. F

In [6]:
# recommended to set it to True when running BigDL in Jupyter notebook. 
OrcaContext.log_output = True # (this will display terminal's stdout and stderr in the Jupyter notebook).

cluster_mode = "local"

if cluster_mode == "local":
    init_orca_context(cores=1, memory="4g") # run in local mode
elif cluster_mode == "k8s":
    init_orca_context(cluster_mode="k8s", num_nodes=2, cores=4) # run on K8s cluster
elif cluster_mode == "yarn":
    init_orca_context(
        cluster_mode="yarn-client", cores=4, num_nodes=2, memory="2g",
        driver_memory="10g", driver_cores=1
        ) # run on Hadoop YARN cluster


Initializing orca context
Current pyspark location is : /home/yina/anaconda3/envs/py37/lib/python3.7/site-packages/pyspark/__init__.py
Start to getOrCreate SparkContext
pyspark_submit_args is:  --driver-class-path /home/yina/anaconda3/envs/py37/lib/python3.7/site-packages/bigdl/share/orca/lib/bigdl-orca-spark_2.4.6-2.1.0-SNAPSHOT-jar-with-dependencies.jar:/home/yina/anaconda3/envs/py37/lib/python3.7/site-packages/bigdl/share/dllib/lib/bigdl-dllib-spark_2.4.6-2.1.0-SNAPSHOT-jar-with-dependencies.jar:/home/yina/anaconda3/envs/py37/lib/python3.7/site-packages/bigdl/share/friesian/lib/bigdl-friesian-spark_2.4.6-2.1.0-SNAPSHOT-jar-with-dependencies.jar:/home/yina/anaconda3/envs/py37/lib/python3.7/site-packages/bigdl/share/core/lib/all-2.1.0-20220314.094552-2.jar pyspark-shell 
[main] WARN  org.apache.spark.util.Utils  - Your hostname, yina-intel resolves to a loopback address: 127.0.1.1; using 10.239.158.177 instead (on interface enp0s31f6)
[main] WARN  org.apache.spark.util.Utils  - Set SP

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-06-13 21:34:25,210 Thread-4 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-06-13 21:34:25,212 Thread-4 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-06-13 21:34:25,212 Thread-4 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-06-13 21:34:25,213 Thread-4 WARN The bufferSize is set to 4000 but bufferedIo is false: false
22-06-13 21:34:25 [Thread-4] INFO  Engine$:121 - Auto detect executor number and executor cores number
22-06-13 21:34:25 [Thread-4] INFO  Engine$:123 - Executor number is 1 and executor cores number is 1
22-06-13 21:34:25 [Thread-4] INFO  ThreadPool$:95 - Set mkl threads to 1 on thread 27
[Thread-4] WARN  org.apache.spark.SparkContext  - Using an existing SparkContext; some configuration may not take effect.
22-06-13 21:34:25 [Thread-4] INFO  Engine$:456 - Find existing spark context. Checking the spark conf...
cls.getname: com.intel.analytics.bigdl.dllib.utils.python.api.Sample
BigDLBasePickler r


User settings:

   KMP_AFFINITY=granularity=fine,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   OMP_NUM_THREADS=1

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=144
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_HAND_THREAD=false
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_FORKJOIN_FRAMES=true
   KMP_FORKJOIN_FRAMES_MODE=3
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_ITT_PREPARE_DELAY=0
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_MWAIT_HINTS=0
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_

This is the only place where you need to specify local or distributed mode. View Orca Context for more details.

**Note**: You should export HADOOP_CONF_DIR=/path/to/hadoop/conf/dir when you run on Hadoop YARN cluster.

### Define the model

In [7]:
class SampleRankingModel(tfrs.models.Model):
    def __init__(self, user_id_num, movie_title_num):
        super().__init__()
        embedding_dim = 32
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )
        self.user_embedding = tf.keras.layers.Embedding(user_id_num + 1, embedding_dim)
        self.movie_embedding = tf.keras.layers.Embedding(movie_title_num + 1, embedding_dim)
        self.ratings = tf.keras.Sequential([
              # Learn multiple dense layers.
              tf.keras.layers.Dense(256, activation="relu"),
              tf.keras.layers.Dense(64, activation="relu"),
              # Make rating predictions in the final layer.
              tf.keras.layers.Dense(1)
          ])

    def call(self, features):
        embeddings = tf.concat([self.user_embedding(features["user_id"]),
                               self.movie_embedding(features["movie_title"])], axis=1)
        return self.ratings(embeddings)

    def compute_loss(self, inputs, training: bool = False) -> tf.Tensor:
        labels = inputs["user_rating"]
        rating_predictions = self(inputs)
        return self.task(labels=labels, predictions=rating_predictions)

### Define the dataset

Use Friesian FeatureTable to get and preprocess the movielens data and split it into a training and test set.

First, we will download the [ml-1m dataset](https://grouplens.org/datasets/movielens/1m/) and unzip it.

In [9]:
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip && unzip ml-1m.zip

--2022-06-13 21:42:39--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving child-prc.intel.com (child-prc.intel.com)... 10.239.120.56
Connecting to child-prc.intel.com (child-prc.intel.com)|10.239.120.56|:913... connected.
Proxy request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.1’

ml-1m.zip.1         100%[===================>]   5.64M  10.6KB/s    in 6m 51s  

2022-06-13 21:49:31 (14.1 KB/s) - ‘ml-1m.zip.1’ saved [5917549/5917549]

Archive:  ml-1m.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of ml-1m.zip or
        ml-1m.zip.zip, and cannot find ml-1m.zip.ZIP, period.


In [11]:
data_dir = "./ml-1m/"

# UserID::MovieID::Rating::Timestamp
# UserID::Gender::Age::Occupation::Zip-code
# MovieID::Title::Genres
dataset = {
    "ratings": ['userid', 'movieid', 'rating', 'timestamp'],
    "movies": ["movieid", "title", "genres"]
}

Then we will use Friesian FeatureTable to read the .dat files.

In [12]:
tbl_dict = dict()
for data, cols in dataset.items():
    tbl = FeatureTable.read_csv(os.path.join(data_dir, data + ".dat"),
                                delimiter=":", header=False)
    tmp_cols = tbl.columns[::2]
    tbl = tbl.select(tmp_cols)
    col_dict = {c[0]: c[1] for c in zip(tmp_cols, cols)}
    tbl = tbl.rename(col_dict)
    tbl_dict[data] = tbl

In [13]:
full_tbl = tbl_dict["ratings"].join(tbl_dict["movies"], "movieid")\
    .dropna(columns=None).select(["userid", "title", "rating"])
full_tbl = full_tbl.cast(["rating"], "int")
full_tbl = full_tbl.cast(["userid"], "string")
full_tbl.show(5, False)

+------+--------------------------------------+------+
|userid|title                                 |rating|
+------+--------------------------------------+------+
|1     |One Flew Over the Cuckoo's Nest (1975)|5     |
|1     |James and the Giant Peach (1996)      |3     |
|1     |My Fair Lady (1964)                   |3     |
|1     |Erin Brockovich (2000)                |4     |
|1     |Bug's Life, A (1998)                  |5     |
+------+--------------------------------------+------+
only showing top 5 rows



Generate unique index value of categorical features and encode these columns with generated string indices.

In [14]:
str_idx = full_tbl.gen_string_idx(["userid", "title"])
user_id_size = str_idx[0].size()
title_size = str_idx[1].size()
full_tbl = full_tbl.encode_string(["userid", "title"], str_idx)
full_tbl.show(5, False)

[Stage 23:========================================>             (151 + 1) / 200]

+------+------+-----+
|rating|userid|title|
+------+------+-----+
|5     |4395  |1855 |
|3     |4395  |136  |
|3     |4395  |2973 |
|4     |4395  |816  |
|5     |4395  |1582 |
+------+------+-----+
only showing top 5 rows



Sample 10% data and split it into a training and test set.

In [15]:
part_tbl = full_tbl.sample(0.1, seed=42)
train_tbl, test_tbl = part_tbl.random_split([0.8, 0.2])

In [16]:
train_count = train_tbl.size()
steps = math.ceil(train_count / 8192)
print("train size: ", train_count, ", steps: ", steps)

test_count = test_tbl.size()
test_steps = math.ceil(test_count / 4096)
print("test size: ", test_count, ", steps: ", test_steps)

train size:  75420 , steps:  10


[Stage 42:>                                                         (0 + 1) / 1]

test size:  18733 , steps:  5


Create Orca TF Datasets from a Friesian FeatureTables.

In [17]:
train_ds = Dataset.from_feature_table(train_tbl)
test_ds = Dataset.from_feature_table(test_tbl)

Once the Orca TF Dataset is created, we can perform some data preprocessing using the map function. Since the model use `input["movie_title"], input["user_id"] and input["user_rating"]` in the model `call` and `compute_loss` function, we should change the key name of the Dataset.

In [18]:
train_ds = train_ds.map(lambda x: {
    "movie_title": x["title"],
    "user_id": x["userid"],
    "user_rating": x["rating"],
})
test_ds = test_ds.map(lambda x: {
    "movie_title": x["title"],
    "user_id": x["userid"],
    "user_rating": x["rating"],
})

Create an Orca Estimator using the SampleRankingModel.

In [19]:
def model_creator(config):
    model = SampleRankingModel(user_id_num=user_id_size, movie_title_num=title_size)
    model = TFRSModel(model)
    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                  optimizer=tf.keras.optimizers.Adagrad(config["lr"]))
    return model

In [20]:
config = {
    "lr": 0.1
}

est = Estimator.from_keras(model_creator=model_creator,
                           verbose=True,
                           config=config, backend="tf2")

E0613 21:54:36.217899060   24654 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0613 21:54:36.315804290   24654 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
2022-06-13 21:54:37,060	INFO services.py:1340 -- View the Ray dashboard at http://10.239.158.177:8265
E0613 21:54:37.103059549   24654 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0613 21:54:37.138308369   24654 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


{'node_ip_address': '10.239.158.177', 'raylet_ip_address': '10.239.158.177', 'redis_address': '10.239.158.177:62454', 'object_store_address': '/tmp/ray/session_2022-06-13_21-54-34_822277_24654/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-06-13_21-54-34_822277_24654/sockets/raylet', 'webui_url': '10.239.158.177:8265', 'session_dir': '/tmp/ray/session_2022-06-13_21-54-34_822277_24654', 'metrics_export_port': 60426, 'node_id': '6c8ec3a3fef337d5614b65e196de5f9298f48f6def7666854d6cf674'}


(pid=25637) /home/yina/anaconda3/envs/py37/lib/python3.7/site-packages/bigdl/dllib/utils/engine.py:47: UserWarning: Find both SPARK_HOME and pyspark. You may need to check whether they match with each other. SPARK_HOME environment variable is set to: /opt/spark-2.4.3-bin-hadoop2.7, and pyspark is found in: /home/yina/anaconda3/envs/py37/lib/python3.7/site-packages/pyspark/__init__.py. If they are unmatched, please use one source only to avoid conflict. For example, you can unset SPARK_HOME and use pyspark only.
(pid=25637)   warnings.warn(warning_msg)
(pid=25637) /home/yina/anaconda3/envs/py37/lib/python3.7/site-packages/bigdl/dllib/utils/zoo_engine.py:47: UserWarning: Find both SPARK_HOME and pyspark. You may need to check whether they match with each other. SPARK_HOME environment variable is set to: /opt/spark-2.4.3-bin-hadoop2.7, and pyspark is found in: /home/yina/anaconda3/envs/py37/lib/python3.7/site-packages/pyspark/__init__.py. If they are unmatched, you are recommended to use 

Then train the model using Orca TF Dataset.

In [21]:
est.fit(train_ds, 3, batch_size=8192, steps_per_epoch=steps)

2022-06-13 21:54:44.507848: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/d1/Dockers/grpcwnd/lib
2022-06-13 21:54:44.507872: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[Stage 60:>                                                         (0 + 1) / 1]2022-06-13 21:54:45.739910: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/d1/Dockers/grpcwnd/lib
2022-06-13 21:54:45.740053: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or 

(Worker pid=25637) Epoch 1/3


(Worker pid=25637) WARNING:tensorflow:AutoGraph could not transform <bound method SampleRankingModel.call of <__main__.SampleRankingModel object at 0x7ef5e67c3cd0>> and will run it as-is.
(Worker pid=25637) Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
(Worker pid=25637) Cause: Unknown node type <gast.gast.Import object at 0x7ef5e64972d0>
(Worker pid=25637) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


10/10 [==============================] - 4s 64ms/step - root_mean_squared_error: 2.1493 - loss: 4.3399 - regularization_loss: 0.0000e+00 - total_loss: 4.3399
(Worker pid=25637) Epoch 2/3
10/10 [==============================] - 1s 70ms/step - root_mean_squared_error: 1.1132 - loss: 1.2575 - regularization_loss: 0.0000e+00 - total_loss: 1.2575
(Worker pid=25637) Epoch 3/3
 7/10 [====================>.........] - ETA: 0s - root_mean_squared_error: 1.1880 - loss: 1.4112 - regularization_loss: 0.0000e+00 - total_loss: 1.4112


[{'train_root_mean_squared_error': 1.1844418048858643,
  'train_loss': 1.3926191329956055,
  'train_regularization_loss': 0.0,
  'train_total_loss': 1.3926191329956055}]

10/10 [==============================] - 0s 35ms/step - root_mean_squared_error: 1.1844 - loss: 1.4020 - regularization_loss: 0.0000e+00 - total_loss: 1.4020


Finally, we can evaluate our model on the test set.

In [22]:
est.evaluate(test_ds, 4096, num_steps=test_steps)

Cause: could not parse the source code of <function <lambda> at 0x7f9ef005f290>: no matching AST found among candidates:

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=26101) /home/yina/anaconda3/envs/py37/lib/python3.7/site-packages/bigdl/dllib/utils/engine.py:47: UserWarning: Find both SPARK_HOME and pyspark. You may need to check whether they match with each other. SPARK_HOME environment variable is set to: /opt/spark-2.4.3-bin-hadoop2.7, and pyspark is found in: /home/yina/anaconda3/envs/py37/lib/python3.7/site-packages/pyspark/__init__.py. If they are unmatched, please use one source only to avoid conflict. For example, you can unset SPARK_HOME and use pyspark only.
(pid=26101)   warnings.warn(warning_msg)
(pid=26101) /home/yina/anaconda3/envs/py37/lib/python3.7/site-packages/bigdl/dllib/utils/zoo_engine.py:47: UserWarning: Find both SPARK_HOME and pyspark. You may need to check whether they match with each other. SPARK_HOME env

1/5 [=====>........................] - ETA: 4s - root_mean_squared_error: 1.6395 - loss: 2.6881 - regularization_loss: 0.0000e+00 - total_loss: 2.6881


[{'validation_root_mean_squared_error': 1.2274852991104126}]

5/5 [==============================] - 1s 25ms/step - root_mean_squared_error: 1.2275 - loss: 1.7887 - regularization_loss: 0.0000e+00 - total_loss: 1.7887


Shutdown the Estimator and stop the orca context.

In [23]:
est.shutdown()
stop_orca_context()

Stopping orca context


2022-06-13 21:55:14,972	ERROR worker.py:1247 -- listen_error_messages_raylet: Connection closed by server.
2022-06-13 21:55:14,975	ERROR import_thread.py:89 -- ImportThread: Connection closed by server.
2022-06-13 21:55:14,978	ERROR worker.py:478 -- print_logs: Connection closed by server.
